# Solutions to Checkpoint 8

## 1. First, make some data preprocessing to clean up the data if you feel necessary. You can use your solution to the assignment of the data preprocessing checkpoint of this module.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import create_engine
import spacy
import markovify


import warnings
warnings.filterwarnings(action="ignore")

!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/sajithgowthaman/opt/anaconda3/lib/python3.7/site-packages/en_core_web_sm
-->
/Users/sajithgowthaman/opt/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'twitter_sentiment'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

tweets_df = pd.read_sql_query('select * from twitter',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [3]:
tweets_df.head()

,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,570306133677760513,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,2,570301083672813571,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)


In [4]:
nlp = spacy.load('en')

# below is necessary to avoid memory error of SpaCy
nlp.max_length = 20000000

# all the processing work is done below, so it may take a while
twitter_doc = nlp(" ".join(tweets_df.text))

## 2. Train a Markov chain model by using only the negative sentiment tweets. Generate some random sentences. Are the generated sentences exhibit the same negative sentiment?

In [5]:
tweets_df[tweets_df["airline_sentiment"]=="negative"].text

3        @VirginAmerica it's really aggressive to blast...
4        @VirginAmerica and it's a really big bad thing...
5        @VirginAmerica seriously would pay $30 a fligh...
15           @VirginAmerica SFO-PDX schedule is still MIA.
17       @VirginAmerica  I flew from NYC to SFO last we...
                               ...                        
14631    @AmericanAir How do I change my flight if the ...
14633    @AmericanAir thx for nothing on getting us out...
14635    @AmericanAir my flight was Cancelled Flightled...
14636    @AmericanAir leaving over 20 minutes Late Flig...
14638    @AmericanAir you have my money, you change my ...
Name: text, Length: 9178, dtype: object

In [6]:
# all the processing work is done below, so it may take a while
# Let's pick the text column for negative tweets of the airline sentiment and document it.
twitter_negative_doc = nlp(" ".join(tweets_df[tweets_df["airline_sentiment"]=="negative"].text))

#Choose sentences with more than 1 character.
tweet_negative_sents = " ".join([sent.text for sent in twitter_negative_doc.sents if len(sent.text) > 1])

tweet_negative_generator = markovify.Text(tweet_negative_sents, state_size = 3)

# three randomly generated negative sentences
for i in range(20):
    print(tweet_negative_generator.make_sentence(tries=100))

# three randomly-generated negative sentences of no more than 100 characters
for i in range(20):
    print(tweet_negative_generator.make_short_sentence(100, tries=100))

SOMEONE PLEASE HELP ME @USAirways gate agent in CLT forced me to check my CC everytime @united Is flight 746 going to sit on the tarmac while they move a plane from DCA to MSY was Cancelled Flightled, so got rebooked to ORD @AmericanAir it's GTSVYB.
@SouthwestAir this is not a good option @united I’ve filled out the form that no one can find.
40 minutes to get anything.
@USAirways it doesn't take 6 days to respond to everyone else's complaints but not mine.
We tried calling customer relations but there is a full day till departure is unacceptable.
@USAirways even if I was there 12 hrs early!
My girlfriend Amy Lloyd is going to charge for a bag w PRIORITY on it?why do u cont to not care for EP's?
Kept us waiting on plane!!! @united again, you said to go take the time to get drunk and tell you how horrible you're attendants are.
Yall petty @JetBlue I just booked a flight yesterday but didnt receive an email confirmation for my ticket, but the credit card was charged on the transaction wh

Clearly the generated tweets show a negative sentiment.

## 3. Do the previous task this time by using only the positive sentiment tweets. Generate some random sentences and observe whether they exhibit positive sentiment or not.

In [7]:
# all the processing work is done below, so it may take a while
twitter_positive_doc = nlp(" ".join(tweets_df[tweets_df["airline_sentiment"]=="positive"].text))

tweet_positive_sents = " ".join([sent.text for sent in twitter_positive_doc.sents if len(sent.text) > 1])

tweet_positive_generator = markovify.Text(tweet_positive_sents, state_size = 3)

# three randomly generated sentences
for i in range(20):
    print(tweet_positive_generator.make_sentence(tries=100))

# three randomly-generated sentences of no more than 100 characters
for i in range(20):
    print(tweet_positive_generator.make_short_sentence(100, tries=100))

#PromoteThatGirl @USAirways Thank you for the tip!
“@JetBlue : Our fleet's on fleek. http://t.co/EUl6sDURbU @JetBlue Gotcha... thanks for the upgrade😉!! so far so good this week.
@JetBlue Love you guys😍😍😍 http://t.co/3X9NRUOvtS @JetBlue you're still the best airline out there no one is better than nothing at all {ahem, @USAirways} @AmericanAir those were snacks we left on a flight today.
I know everyone is working hard to get us out of the snow.
@USAirways Ann Marie at LGA is the best part of flying!
It would mean LUV is in the air. http://t.co/YNc2ZUt4zz @SouthwestAir thanks!
@JetBlue Thanks for the follow up.
@JetBlue Worked now, ty @JetBlue I would go anywhere JetBlue goes.
#usairways http://t.co/Gt6umHbh43 @AmericanAir thank you @AmericanAir simply amazing.
IAH to DFW to OKC has turned out to be a member of the #inflight crew team...
@AmericanAir Thank you so much!
Took care of it when I called @united awesome thank you very very much!! 💙💙 @JetBlue Awww thank you B6!
NO to @Carrieu

This time we observe positive sentiments generally.

## 4. This time train your Markov chain model using all the tweets and generate some random sentences. Can you say something systematic about the sentiments of the generated tweets?

In [8]:
# all the processing work is done below, so it may take a while
twitter_doc = nlp(" ".join(tweets_df.text))

tweet_sents = " ".join([sent.text for sent in twitter_doc.sents if len(sent.text) > 1])

tweet_generator = markovify.Text(tweet_sents, state_size = 3)

# three randomly generated sentences
for i in range(20):
    print(tweet_generator.make_sentence(tries=100))

# three randomly-generated sentences of no more than 100 characters
for i in range(20):
    print(tweet_generator.make_short_sentence(100, tries=100))

Still need to improve service # united @united perhaps you could scale your excitement back by a few weeks...
@united once again my bag is in San Jose.
Am I to sleep in a toddler bed at the airport with the appropriate amount of time for the wife and my 15th anniversary!
2+ hour hold times are long can I expect the formal response to take?
Is that something I can do to help?
Going through multiple prompts &amp; menus just to be told I need to Cancelled Flight my flight.
Fingers crossed we get into the disaster that was checking bags.
@JetBlue I'd go to Portsmouth, NH to take care of @LSUsoftball team &amp; @LSUQuinlanDuhon team.
But more than two hours Late Flight and gets on to PHL @USAirways the link doesn't work..
Is there any chance I could bump up my flight time and the attendants at MKE made my day!
Auto rebooked to Tuesday??? Then 3hr 26min on hold for almost an hour, waiting to make a ticket change via Twitter?
2 hr delay is a vacation buzzkill @united just touched down in #New

We can see both positive and negative sentiments this time.